In [ ]:
!pip install torch torchvision torchaudio
!pip install matplotlib numpy pillow opencv-python scikit-learn
!pip install torchsummary tensorboard

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Mount Google Drive to save/load models
from google.colab import drive
drive.mount('/content/drive')

# Create a directory for your project
import os
project_dir = '/content/drive/MyDrive/NeuroScale_AlexNet'
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuroScaleAlexPlusPlus(num_classes=10, input_channels=3)
model = model.to(device)

print(f"Model loaded on: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
import torch.nn as nn

class NeuroScaleAlexPlusPlus(nn.Module):
    def __init__(self, num_classes=10, input_channels=3):
        super(NeuroScaleAlexPlusPlus, self).__init__()
        # Define your model layers here
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Load CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        if batch_idx % 100 == 0:
            print(f'Batch [{batch_idx}/{len(dataloader)}], Loss: {loss.item():.4f}')

    return running_loss/len(dataloader), 100.*correct/total

In [ ]:
num_epochs = 50
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    print(f'\nEpoch [{epoch+1}/{num_epochs}]')

    # Train
    train_loss, train_acc = train_epoch(model, trainloader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Update learning rate
    scheduler.step()

    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')

    # Save checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
        }, f'checkpoint_epoch_{epoch+1}.pth')

print("Training completed!")


Epoch [1/50]
Batch [0/1563], Loss: 2.3004
Batch [100/1563], Loss: 2.0884
Batch [200/1563], Loss: 2.1748
Batch [300/1563], Loss: 2.1798
Batch [400/1563], Loss: 1.8987
Batch [500/1563], Loss: 2.0095
Batch [600/1563], Loss: 2.1949
Batch [700/1563], Loss: 1.9490
Batch [800/1563], Loss: 1.7234
Batch [900/1563], Loss: 1.4179
Batch [1000/1563], Loss: 1.7574
Batch [1100/1563], Loss: 1.7020
Batch [1200/1563], Loss: 1.6521
Batch [1300/1563], Loss: 1.6787
Batch [1400/1563], Loss: 1.7590
Batch [1500/1563], Loss: 1.6594
Train Loss: 1.8162, Train Acc: 31.66%

Epoch [2/50]
Batch [0/1563], Loss: 1.7761
Batch [100/1563], Loss: 1.5811
Batch [200/1563], Loss: 1.6108
Batch [300/1563], Loss: 1.5535
Batch [400/1563], Loss: 1.6834
Batch [500/1563], Loss: 1.4400
Batch [600/1563], Loss: 1.4250
Batch [700/1563], Loss: 1.5035
Batch [800/1563], Loss: 1.6691
Batch [900/1563], Loss: 1.5451
Batch [1000/1563], Loss: 1.8352
Batch [1100/1563], Loss: 1.4901
Batch [1200/1563], Loss: 1.5631
Batch [1300/1563], Loss: 1.568

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt

def test_user_image():
    # Upload image
    print("Please upload an image:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        # Load and preprocess image
        image = Image.open(filename).convert('RGB')

        # Display original image
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(image)
        plt.title('Original Image')
        plt.axis('off')

        # Preprocess for model
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Make prediction
        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = torch.softmax(outputs, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1)
            confidence = torch.max(probabilities).item()

        classes = [
    'Healthy',
    'Leaf Blight',
    'Bacterial Spot',
    'Early Blight',
    'Late Blight',
    'Leaf Curl',
    'Powdery Mildew',
    'Rust',
    'Yellow Mosaic',
    'Nutrient Deficiency'
]


        # Display results
        plt.subplot(1, 3, 2)
        plt.bar(classes, probabilities[0].cpu().numpy())
        plt.title('Class Probabilities')
        plt.xticks(rotation=45)

        plt.subplot(1, 3, 3)
        plt.text(0.1, 0.8, f'Predicted: {classes[predicted_class.item()]}', fontsize=14)
        plt.text(0.1, 0.6, f'Confidence: {confidence:.2%}', fontsize=14)
        plt.text(0.1, 0.4, f'NeuroScale-Alex++Net Analysis', fontsize=12, style='italic')
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.axis('off')
        plt.title('Prediction Results')

        plt.tight_layout()
        plt.show()

        return predicted_class.item(), confidence

In [ ]:
test_user_image()